In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
from scipy.signal import savgol_filter
from six.moves import xrange
import umap
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import make_grid


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
#加载Cifir-10
training_set = datasets.CIFAR10(root="data",train=True,download=True,transform=transforms.Compose([
    transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(1,1,1))
]))

validation_set = datasets.CIFAR10(root="data",train=False,download=True,transform=transforms.Compose([
    transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(1,1,1))
]))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting data\cifar-10-python.tar.gz to data
Files already downloaded and verified


In [ ]:
#VQ 层
class VectorQuantizer(nn.Module):
    def __init__(self,num_channels:int,embedding_dim:int,commitment_cost:float):
        super.__init__()
        self._embedding_dim  = embedding_dim
        self._num_channels = num_channels

        self._embedding = nn.Embedding(self._num_channels,self._embedding_dim)
        # 给这个可学习的权重初始化，直接用这个权重作为Codebook了。这个weight: [numc,embeddim].
        self._embedding.weight.data.uniform_(-1/self._num_channels,1/self._num_channels)
        self._commitment_cost = commitment_cost



    def forward(self,inputs:torch.Tensor):
        inputs = inputs.permute(0,2,3,1).contiguous() #内存数据连续化
        input_shape = inputs.shape

        # 拉成二维
        flat_input = inputs.view(-1,self._embedding_dim)

        #计算距离 这里使用的事欧式距离。也是是余弦相似度。其实之间是有关系的。
        distance = (torch.sum(flat_input**2,dim=1,keepdim=True)+
                    torch.sum(self._embedding.weight**2,dim=1)-
                    2*torch.matmul(flat_input,self._embedding_dim()))

        #Ecoding
        encoding_indices = torch.argmin(distance,dim=1).unsqueeze(1)
        encodings = torch.zeros(encoding_indices.shape[0],self._num_channels,device=inputs.device)
        encodings.scatter_(1,encoding_indices,1)























